# Item Based Recommendation 

Recommendation based on track similarity. A track is similar to another one if it is listened by the same user.

In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
# read test set
training = pd.read_csv("training.csv")[['user', 'track']]
training.head()

,user,track
0,000c556aa021426138c47076d53dfbec,spotify:track:000UQuSa2QcsWFK0tGTBDM
1,000c556aa021426138c47076d53dfbec,spotify:track:001faoWrA2TK6dw9vHp9p6
2,000c556aa021426138c47076d53dfbec,spotify:track:010jenmHogRDArWKod52XR
3,000c556aa021426138c47076d53dfbec,spotify:track:01gijlGB36OqjXJ0bUkE99
4,000c556aa021426138c47076d53dfbec,spotify:track:01z7U4QxORx59DIfSwImo2


In [15]:
# read test set
tracks = training['track'].unique()
size = len(tracks)
items = np.zeros((size,size))
track_id_dict = {}
id_track_dict = {}
for i,track in enumerate(tracks):
    track_id_dict[i] = track
    id_track_dict[track] = i

In [16]:
user_dict = {}
groups = training.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict[user_id] = track_ids    

In [19]:
count = 0
for tracks in user_dict.values():
    track_ids = [id_track_dict[track] for track in tracks]
    sys.stdout.write("User: "+str(count)+" --- "+str(len(track_ids))+" tracks \r")
    sys.stdout.flush()
    count += 1
    if len(track_ids) < 500:
        for track_id in track_ids:
            items[track_id,track_ids] += 1

## Suche

In [22]:
track_list = ['spotify:track:000UQuSa2QcsWFK0tGTBDM', 'spotify:track:01gijlGB36OqjXJ0bUkE99']
id_list = [id_track_dict[track] for track in track_list]

In [35]:
result = np.zeros((size, ))
for track_id in id_list:
    result += items[track_id]
result = [(index, item) for index, item in enumerate(result)]
result.sort(key=lambda item: item[1], reverse=True)
result = result[:10]
result = [(track_id_dict[i[0]]) for i in result]
result

['spotify:track:000UQuSa2QcsWFK0tGTBDM',
 'spotify:track:001faoWrA2TK6dw9vHp9p6',
 'spotify:track:010jenmHogRDArWKod52XR',
 'spotify:track:01gijlGB36OqjXJ0bUkE99',
 'spotify:track:01z7U4QxORx59DIfSwImo2',
 'spotify:track:02lu50t1jrndVEOxebVd3Y',
 'spotify:track:038TIbMNhiU691wSbikNBH',
 'spotify:track:03bVBb42usKDSDX9HaypQP',
 'spotify:track:03KIUGxW3RA4U7YoXvIxYK',
 'spotify:track:03uLxw86NqtmPoES1xqg8L']